In [212]:
import numpy as np
from faker import Faker
import pandas as pd
import os
from datetime import datetime
import random

fake = Faker()

In [167]:
def erstelle_fake_data(num = 10, seed = 123):
    np.random.seed(seed)
    fake.seed_instance(seed)
    output = []
    for x in range(num):
        geschlecht = np.random.choice(["M", "W"], p=[0.65, 0.35])
        output.append(
            {
                "Name": fake.first_name_male() if geschlecht=="M" else fake.first_name_female(),
                "Hautfarbe": np.random.choice(["White", "Black"], p=[0.4, 0.6]),
                "Laufende_Strafe": np.random.choice([1,2,3,4,5]),
                "Geschlecht": geschlecht,
                "Haerte_des_Vergehens": np.random.choice(["Leicht", "Mittel", "Hart"], p=[0.45, 0.35, 0.2])
            }  
        )
    return output

In [168]:
def regeln_erstellen(antrag_werte, antrag_bias, bias):
    regel_pos = {}
    regel_neg = {}
    regel_neg_bias = {}
    regel_pos_bias = {}
    for key in antrag_werte.keys():
        values = antrag_werte[key]
        length = int(len(values)/2)+1
        pos_values = []
        for x in range(length):
            pos_values.append(values[x])
        regel_pos[key] = pos_values
        neg_values = []
        for y in range(len(values)):
            if(y >= length):
                neg_values.append(values[y])
        regel_neg[key] = neg_values
    regel_pos_bias = regel_pos.copy()
    regel_neg_bias = regel_neg.copy()
    if(bias != None):
        for key in antrag_bias.keys():
            if(bias.keys().__contains__(key)):
                regel_neg_bias[key] = bias[key]
                neg_bias = bias[key]
                regel_pos_bias[key] = []
                for val in antrag_bias[key]:
                    if not(neg_bias.__contains__(val)):
                        regel_pos_bias[key].append(val)
    return(regel_pos, regel_neg, regel_pos_bias, regel_neg_bias)

In [169]:
def seed_generieren():
    now = datetime.now()
    seed = (now.day * now.minute * now.second * now.month * now.year * now.hour) / now.microsecond 
    print(seed)
    if(seed <= 0):
        seed = now.day * (now.minute + 1)
    return seed

In [ ]:
class Bewerter:
    def __init__(self, regeln_pos, regeln_neg):
        self.regeln_pos = regeln_pos
        self.regeln_neg = regeln_neg
    def bewerte(self, antrag):
        for key in self.regeln_pos.keys():
            if not(self.regeln_pos[key].__contains__(antrag[key])):
                antrag["Bewertung"] = "negativ"
                return antrag
        antrag["Bewertung"] = "positiv"
        return antrag

In [215]:
def work(df, bias, anzahlBewerter):
    antrag_füllwerte = {
        "Name": "random"
    }
    antrag_werte = {
        "Laufende_Strafe": [1,2,3,4,5],
        "Haerte_des_Vergehens": ["Leicht", "Mittel", "Hart"]
    }
    antrag_bias = {
        "Hautfarbe": ["Black", "White"],
        "Geschlecht": ["M", "W"]
    }
    r_pos, r_neg, rb_pos, rb_neg = regeln_erstellen(antrag_werte, antrag_bias, bias)
    #Bewerten
    bewerter = []
    for x in range(anzahlBewerter):
        bewerter.append(Bewerter(regeln_pos=r_pos, regeln_neg=r_neg))
    biasBewerter = round(anzahlBewerter * 0.2)
    if(biasBewerter == 0):
        biasBewerter = 1
    for x in range(biasBewerter):
        bewerter[x].regeln_pos = rb_pos
        bewerter[x].regeln_neg = rb_neg
    i = 0
    fertige_antraege = []
    for index, r in df.iterrows():
        #if(i >= anzahlBewerter):
        #    i = 0
        i = random.randint(0, anzahlBewerter-1)
        antrag = bewerter[i].bewerte(r.copy())
        fertige_antraege.append(antrag)
        #i += 1
    newdf = pd.DataFrame(fertige_antraege)
    return newdf

    
def daten_generieren(anzahlDaten):
    seed = seed_generieren()
    df = pd.DataFrame(erstelle_fake_data(anzahlDaten,int(seed)))
    return df

In [231]:
bias = {
    "Hautfarbe": ["Black"]
}
data = daten_generieren(10)
print(data)
finished = work(data,None,5)
print(finished)
data.to_csv("Daten.csv", sep=';', encoding='utf-8', index=False)

411.89273249846025
        Name Hautfarbe  Laufende_Strafe Geschlecht Haerte_des_Vergehens
0      Cindy     Black                3          W               Mittel
1    Gabriel     Black                1          M               Mittel
2      Larry     Black                5          M               Mittel
3      David     White                1          M               Leicht
4  Stephanie     Black                3          W               Leicht
5      James     Black                5          M               Mittel
6     Daniel     White                4          M               Leicht
7       Adam     Black                5          M               Mittel
8    Timothy     Black                1          M               Mittel
9    Michael     Black                5          M               Mittel
        Name Hautfarbe  Laufende_Strafe Geschlecht Haerte_des_Vergehens  \
0      Cindy     Black                3          W               Mittel   
1    Gabriel     Black                1